ELAN to CSV for multiple files at once

In [ ]:
import os
import re
import csv
import glob
from bs4 import BeautifulSoup as bs

#dir_path = "C:\\Users\\barth\\Dropbox\\CoE-LDaCA\\AccentedAustralianEnglish\\Transcripts_Anonymised\\test\\"
#dir_path = "C:\\Users\\barth\\Dropbox\\CoE-LDaCA\\AccentedAustralianEnglish\\Transcripts_Anonymised\\"
#dir_path = "C:/Users/barth/Dropbox/CoE_SydSpks_anonymised/Sydney-Speaks/SSDS_anon/elan/"
dir_path = 'C:/Users/barth/Documents/LDACA\data_cleaning/AAE/AusESL_internal_after_doublespace/'

print (dir_path)
files = glob.glob(dir_path + "*.eaf")
print(files)
print ("+++ FILES LOADED: ", len(files))

itterate over all files and read the XML tags into a CSV file

A filter function removes all unwanted tiers

In [ ]:
def time_value_to_hms(time_value):
    # trasforms the millisec int into hh:mm:ss:mmmm string
    millis= time_value
    millis = int(millis)
    time_value_padded =  "000" + time_value
    mill = int(time_value_padded[-3:])
    print (mill)
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))%60
    minutes = int(minutes)
    hours=(millis/(1000*60*60))%24
    return ("%d:%d:%d.%d" % (hours, minutes, seconds, mill))

def time_slot_dict_maker(soup):
        # create the time slot reference dict
        timeSlotDict = {}
        time_slots = soup.findAll("time_slot")
        
        for time_slot in time_slots:
            time_value = time_value_to_hms(time_slot["time_value"])
            timeSlotDict[time_slot["time_slot_id"]] = time_value
        
        return timeSlotDict

def write_to_file(outfile, fileList):
    with open(outfile, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(fileList)

def alignable_annotation():
    
    annotation_id = annotation['annotation_id']
    time_slot_ref1 = annotation['time_slot_ref1']
    time_slot_ref2 = annotation['time_slot_ref2']
    text = annotation.annotation_value.get_text()

    annotationRefDict[annotation_id] = [time_slot_ref1, time_slot_ref2]

    return [filename,
            annotation_id.replace("a", ""),
            timeSlotDict[time_slot_ref1],
            timeSlotDict[time_slot_ref2],
            tier_id,
            text]

def ref_annotation():
    
    annotation_id = annotation['annotation_id']
    annotation_ref = annotation['annotation_ref']
    if 'prefious_annotation' in annotation:
        previous_annotation = annotation['previous_annotation']
    text = annotation.annotation_value.get_text()

    time_slot_ref1 = annotationRefDict[annotation_ref][0]
    time_slot_ref2 = annotationRefDict[annotation_ref][1]

    annotationRefDict[annotation_id] = [time_slot_ref1, time_slot_ref2]


    return [filename,
            annotation_id.replace("a", ""),
            timeSlotDict[time_slot_ref1],
            timeSlotDict[time_slot_ref2],
            tier_id,
            text]

def filter_list(fileList):
    # only keep the tiers you want
    filteredList = [headers]
    for each in fileList:
        speaker = each[4]
        if speaker.startswith("INT: "):
            filteredList.append(each)
        elif speaker.startswith("LaBB-CAT_PNT: "):
            filteredList.append(each)
    return filteredList
    
headers = ['Transcript', 
           'IU_number', 
           'start_time', 
           'end_time', 
           'speaker', 
           'IU']

outList = []

for file in files:

    filename = os.path.basename(file)
    outfile = dir_path + filename[:-4] +".csv"
    
    with open(file, "r", encoding="utf-8") as f:
        
        annotationRefDict = {}
        fileList = []
        fileList.append(headers)
        
        soup = bs(f, "lxml")
        
        timeSlotDict = time_slot_dict_maker(soup)

        tiers = soup.find_all('tier')
        for tier in tiers:
            
            #tier level data
            linguistic_type_ref = tier['linguistic_type_ref']
            tier_id = tier['tier_id']
            
            # annotation level data
            # there are two types of annotation tags:
            
            for annotation in tier.find_all('alignable_annotation'):
                tierList = alignable_annotation()
                fileList.append(tierList)
                
            for annotation in tier.find_all('ref_annotation'):
                tierList = ref_annotation()
                fileList.append(tierList)
    
    # only include the tiers you want
    filteredList = filter_list(fileList)
   
    print(str(len(fileList)) + " filtered to: " + str(len(filteredList)) + "\t" +filename)
    write_to_file(outfile, filteredList) 
    print(" +++ DONE +++")